# Lab 7.2 Fine-Tuning

In this notebook you will explore fine-tuning a CNN to classify pet breeds.

Here is some code to download a prepared version of the [Oxford-IIIT Pet dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/).

In [1]:
import os
if not os.path.exists('oxford_pets.zip'):
  !wget "https://www.dropbox.com/scl/fi/p49ifha27c2u3uptfj42w/oxford_pets_corrected.zip?rlkey=dwk3dsptzir8v846imsq6bgw3&dl=1" -O oxford_pets.zip
  !unzip -qq oxford_pets.zip

--2024-11-19 02:53:37--  https://www.dropbox.com/scl/fi/p49ifha27c2u3uptfj42w/oxford_pets_corrected.zip?rlkey=dwk3dsptzir8v846imsq6bgw3&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc3febd5c619acab7f1578dda71.dl.dropboxusercontent.com/cd/0/inline/CeqZ45QyK9RB2kVdit_L-03u3_fTjSieOIrWFYEdrCAmqBWusdkajpXLWBNEWm9NKthtcVIEcvVVWKphOy-KlHz4QmgOWBc29qapC-qE8O1tmkdVr-aBqgtAfWP-keAornCCcNhKHcy-KBhuSckH2nSD/file?dl=1# [following]
--2024-11-19 02:53:38--  https://ucc3febd5c619acab7f1578dda71.dl.dropboxusercontent.com/cd/0/inline/CeqZ45QyK9RB2kVdit_L-03u3_fTjSieOIrWFYEdrCAmqBWusdkajpXLWBNEWm9NKthtcVIEcvVVWKphOy-KlHz4QmgOWBc29qapC-qE8O1tmkdVr-aBqgtAfWP-keAornCCcNhKHcy-KBhuSckH2nSD/file?dl=1
Resolving ucc3febd5c619acab7f1578dda71.dl.dropboxusercontent.com (ucc3febd5c619acab7f1578dda71.dl.dropboxuserconte

In [13]:
import keras
import numpy as np
from matplotlib import pyplot as plt
from keras import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Lambda
from keras.regularizers import L2
from tensorflow.keras.layers import RandomFlip, RandomZoom, RandomRotation
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16

In [4]:
!ls oxford_pets

test  train


### Exercises

1. Set up the data loaders.

- Use `keras.utils.image_dataset_from_directory` to create training, validation and test datasets.  
- Set the image size to (224,224) and use a validation split of 0.1.
- Remember to use the same random seed for training and validation splits.

In [8]:
train_ds = keras.preprocessing.image_dataset_from_directory('oxford_pets/train',
                                                            image_size=(224,224),
                                                            validation_split=0.1,
                                                            subset='training',
                                                            seed=27)

Found 6639 files belonging to 35 classes.
Using 5976 files for training.


In [9]:
val_ds = keras.preprocessing.image_dataset_from_directory('oxford_pets/train',
                                                            image_size=(224,224),
                                                            validation_split=0.1,
                                                            subset='validation',
                                                            seed=27)

Found 6639 files belonging to 35 classes.
Using 663 files for validation.


In [22]:
test_ds = keras.preprocessing.image_dataset_from_directory('oxford_pets/test',
                                                            image_size=(224,224),
                                                            seed=27)

Found 739 files belonging to 35 classes.


2. Create a `Sequential` model with the pre-trained `VGG16` network.

The model should have the following layers:
- Input layer
- One or more [data augmentation](https://keras.io/api/layers/preprocessing_layers/image_augmentation/) layers such as `RandomFlip`, `RandomZoom`, `RandomRotation`, etc.
- VGG16 preprocess function inside a `Lambda` layer
- VGG16 layer: don't include top; use `max` pooling.
- Dense layer with 35 outputs and correct activation function for multi-class classification

In [15]:
model = Sequential([
    Input(shape=(224,224,3)), #0
    RandomZoom(.2),#1
      # Preprocessing layer for VGG16
    Lambda(preprocess_input),#2

    # VGG16 base model
    VGG16(include_top=False, weights='imagenet', pooling='max'), #3

    # Dense output layer
    Dense(35, activation='softmax') #4
])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Set the VGG16 part of the network to be fixed by setting the `trainable` attribute of VGG16 layer to `False`.  (You can access the layers of the model with `model.layers`.)

In [16]:
model.layers[3].trainable = False  # VGG16 is the 4th layer in the Sequential model

Check the model summary to make sure that the VGG16 layer is trainable (it should report a very large number of non-trainable parameters, and a small number of trainable parameters.)


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ random_zoom (RandomZoom)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 512)                 │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 35)                  │          17,955 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,732,643 (56.20 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 17,955 (70.14 KB)

Compile the model with multi-class classification loss and accuracy metric.  You can use Adam with learning rate 3e-4.

In [26]:
model.compile(optimizer=Adam(3e-4),
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

4. Evaluate the model on the test set and check that the accuracy is about $1/35=.029$.

In [27]:
model.evaluate(test_ds)

24/24 ━━━━━━━━━━━━━━━━━━━━ 10s 189ms/step - accuracy: 0.0265 - loss: 161.6640


[162.3706512451172, 0.025710418820381165]

5. Now train the model on the training set (don't forget to include the validation set) for 20 epochs.

In [28]:
model.fit(train_ds, validation_data=val_ds, epochs=20)

Epoch 1/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 107s 512ms/step - accuracy: 0.0280 - loss: 8.6304 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 2/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 85s 452ms/step - accuracy: 0.0256 - loss: 3.5554 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 3/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 142s 453ms/step - accuracy: 0.0264 - loss: 3.5554 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 4/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 142s 453ms/step - accuracy: 0.0263 - loss: 3.5554 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 5/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 142s 453ms/step - accuracy: 0.0246 - loss: 3.5554 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 6/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 142s 452ms/step - accuracy: 0.0266 - loss: 3.5554 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 7/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 144s 462ms/step - accuracy: 0.0248 - loss: 3.5554 - val_accuracy: 0.0226 - val_loss: 3.5553
Epoch 8/20
187/187 ━━━━━━━━━━━━━━━━━━━━ 140s 453ms/step - accuracy: 0.0252 - 

6. Evaluate the accuracy of the fine-tuned model on the test set.

In [29]:
model.evaluate(test_ds)

24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 148ms/step - accuracy: 0.0237 - loss: 3.5553


[3.5553481578826904, 0.027063598856329918]

7. The following code will show some test images with the correct and predicted labels.

In [ ]:
images, labels = next(iter(test_ds))
preds = model.predict(images)

In [ ]:
for im,label,pred in zip(images,labels,preds):
  plt.imshow(im.numpy().astype('uint8'))
  plt.title(f'correct: {test_ds.class_names[label]} | predicted: {test_ds.class_names[np.argmax(pred)]}')
  plt.show()